# Degeneracy Hunter Examples
<br/> <br/>
This notebook shows how to use the following Degeneracy Hunter features through several motivating examples:
* Add features here

## Conventions
The typographic conventions used in this tutorial are as follows.

Names of files will be in "double quotes", new terms will be in *italics*, and keywords and source code snippets will be in `fixed-width type`.

Sections that offer additional detail that may be skipped are set off in blocks like this:
<div class="alert alert-block alert-info">
    &#9998; <b>Details:</b>
    Detailed information goes here...
</div>
   

## Example 1: Linear Program with Equality Constraints
